# Building the model

The purpose of this notebook is to build the model that will eventually be used as the model behind the PL prediction for the app, and building the script to rebuild the model when new player data is introduced weekly

## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from scikeras.wrappers import KerasRegressor
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from joblib import dump, load

## Loading the dataset

In [2]:
combined = pd.read_csv("../files/final_combined_dataframe.csv")

In [3]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2116 entries, 0 to 2115
Data columns (total 94 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   match_id                                2116 non-null   object 
 1   goals                                   2116 non-null   float64
 2   assists                                 2116 non-null   float64
 3   direct_goal_contributions               2116 non-null   float64
 4   non_penalty_goals                       2116 non-null   float64
 5   penalties_scored                        2116 non-null   float64
 6   penalties_attempted                     2116 non-null   float64
 7   yellow_cards                            2116 non-null   float64
 8   red_cards                               2116 non-null   float64
 9   expected_goals                          2116 non-null   float64
 10  non_penalty_expected_goals              2116 non-null   floa

In [4]:
combined.head()

,match_id,goals,assists,direct_goal_contributions,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,...,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards,home_team_at_home_mean_goal_difference,home_team_overall_mean_goal_difference,away_team_at_away_mean_goal_difference,away_team_overall_mean_goal_difference,head_to_head_goal_difference
0,m-00381,0.393401,0.269112,0.662513,0.355397,0.038004,0.054601,0.556054,-0.031945,0.394375,...,9,2,2,0,0,2.4,1.6,-2.8,0.2,7
1,m-00382,0.134401,0.408515,0.542916,0.169614,-0.035212,-0.062705,0.464746,0.040671,0.203698,...,14,5,2,0,0,1.4,1.6,-0.6,1.2,1
2,m-00383,1.032333,0.289712,1.322045,0.976505,0.055828,0.080853,0.654894,-0.011413,0.903525,...,14,2,5,0,0,1.0,1.6,-1.4,0.2,8
3,m-00384,0.073795,0.225465,0.299260,0.152876,-0.079081,-0.080597,0.105934,-0.043515,-0.051271,...,15,0,2,0,0,2.0,1.6,0.0,0.6,1
4,m-00385,-0.308318,-0.074280,-0.382598,-0.265916,-0.042402,-0.029883,0.243744,-0.032462,-0.213221,...,12,4,1,0,0,1.8,1.6,1.0,0.0,3


## Column Lists

In [5]:
output_columns = [
	"home_goals", "away_goals", "home_shots", "away_shots", "home_shots_on_target", "away_shots_on_target",
	"home_corners", "away_corners", "home_fouls", "away_fouls", "home_yellow_cards", "away_yellow_cards",
	"home_red_cards", "away_red_cards"
]
match_columns = [
	"match_id", "competition_id", "home_team_id", "away_team_id", "referee_id",
	"home_goals", "away_goals", "home_shots", "away_shots", "home_shots_on_target", "away_shots_on_target",
	"home_corners", "away_corners", "home_fouls", "away_fouls", "home_yellow_cards", "away_yellow_cards",
	"home_red_cards", "away_red_cards"
]
stats_columns = [
	"goals","assists","penalties_scored","penalties_attempted","yellow_cards","red_cards","expected_goals",
	"non_penalty_expected_goals","expected_assisted_goals","progressive_carries","progressive_passes","total_passing_distance",
	"total_progressive_passing_distance","short_passes_completed","short_passes_attempted","medium_passes_completed",
	"medium_passes_attempted","long_passes_completed","long_passes_attempted","expected_assists","key_passes",
	"passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","shots","shots_on_target",
	"average_shot_distance","shots_from_free_kicks","touches_in_defensive_penalty_area","touches_in_defensive_third",
	"touches_in_middle_third","touches_in_attacking_third","touches_in_attacking_penalty_area","live_ball_touches",
	"take_ons_attempted","take_ons_succeeded","carries","total_carrying_distance","progressive_carrying_distance",
	"carries_into_final_third","carries_into_penalty_area","miscontrols","dispossessed","passes_received",
	"progressive_passes_received","tackles_won","defensive_third_tackles","middle_third_tackles","attacking_third_tackles",
	"dribblers_tackled","dribbler_tackles_attempted","shots_blocked","passes_blocked","interceptions","clearances",
	"errors_leading_to_shot","goals_against","shots_on_target_against","saves","clean_sheets","penalties_faced",
	"penalties_allowed","penalties_saved","penalties_missed"
]
player_stats_columns = ["player_id", "minutes_played","ninetys"] + stats_columns
pure_stats_columns = ["minutes_played"] + stats_columns
team_stats_columns = ["team_id"] + stats_columns


## Scaling

In [6]:
scaler = StandardScaler(copy=True)

In [7]:
combined_standardized = scaler.fit_transform(combined[stats_columns])

In [8]:
evr = {}
n_components = {}
feature_to_pc = {}

for n in range(1,26):
	pca = PCA(n_components = n, random_state=938)
	pca.fit(combined_standardized)
	feature_to_pc_map = pd.DataFrame(pca.components_, columns=stats_columns)
	components = pca.transform(combined_standardized)
	components_df = pd.DataFrame(data=components[:, [p for p in range(n)]], columns=pca.get_feature_names_out(), )
	
	evr[n] = sum(pca.explained_variance_ratio_)
	n_components[n] = components_df
	feature_to_pc[n] = feature_to_pc_map

pd.DataFrame(data=evr, index=["explained_variance_ratio"]).T
feature_to_pc[15].to_csv("../files/feature_to_15_pcs.csv")


Check the effect of different numbers if PCs on the outcome of the NN, testing n=2, 5 and 10. Use each n components to train and test the neural network, and compare the performance of each one

## Number of Features

In [9]:
INPUT_SIZE = len(stats_columns)
OUTPUT_SIZE = 14
ROWS = 1927

### n = 2

Train test split of data

In [10]:
n=2
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

Model definition

In [11]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(len(X_train.columns), activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500


2024-02-21 01:08:33.185389: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


34/34 [==============================] - 0s 713us/step - loss: 41.0263 - accuracy: 0.4450
Epoch 2/500
34/34 [==============================] - 0s 516us/step - loss: 26.6967 - accuracy: 0.4740
Epoch 3/500
34/34 [==============================] - 0s 531us/step - loss: 12.8823 - accuracy: 0.4740
Epoch 4/500
34/34 [==============================] - 0s 453us/step - loss: 9.0366 - accuracy: 0.4740
Epoch 5/500
34/34 [==============================] - 0s 428us/step - loss: 8.7446 - accuracy: 0.4740
Epoch 6/500
34/34 [==============================] - 0s 431us/step - loss: 8.8324 - accuracy: 0.4740
Epoch 7/500
34/34 [==============================] - 0s 424us/step - loss: 8.7403 - accuracy: 0.4740
Epoch 8/500
34/34 [==============================] - 0s 403us/step - loss: 8.5990 - accuracy: 0.4740
Epoch 9/500
34/34 [==============================] - 0s 440us/step - loss: 8.6086 - accuracy: 0.4740
Epoch 10/500
34/34 [==============================] - 0s 436us/step - loss: 8.7018 - accuracy: 0.474

In [12]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 0s 414us/step
[ 1.2126752   1.6975286  11.337424   13.793308    3.9474387   5.082057
  4.645624    5.4389553  10.37679    10.253022    1.7802657   1.5439035
  0.04401858  0.05191425]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1634,2,1,13,16,4,3,3,5,8,10,1,1,0,0
1479,2,4,16,15,6,9,3,3,13,19,6,3,0,0
25,2,1,6,23,2,6,4,8,14,11,4,2,0,0
1686,2,1,12,17,4,5,6,8,11,7,3,1,0,0
1454,1,3,9,15,3,7,3,8,8,15,0,2,0,0


### n = 5

In [13]:
n=5
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [14]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(len(X_train.columns), activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
34/34 [==============================] - 0s 490us/step - loss: 28.6081 - accuracy: 0.4178
Epoch 2/500
34/34 [==============================] - 0s 459us/step - loss: 16.5957 - accuracy: 0.4740
Epoch 3/500
34/34 [==============================] - 0s 438us/step - loss: 12.0450 - accuracy: 0.4740
Epoch 4/500
34/34 [==============================] - 0s 839us/step - loss: 9.9899 - accuracy: 0.4740
Epoch 5/500
34/34 [==============================] - 0s 545us/step - loss: 9.0152 - accuracy: 0.4900
Epoch 6/500
34/34 [==============================] - 0s 408us/step - loss: 8.4597 - accuracy: 0.5296
Epoch 7/500
34/34 [==============================] - 0s 392us/step - loss: 8.1596 - accuracy: 0.5437
Epoch 8/500
34/34 [==============================] - 0s 410us/step - loss: 8.1780 - accuracy: 0.5414
Epoch 9/500
34/34 [==============================] - 0s 509us/step - loss: 7.7837 - accuracy: 0.5461
Epoch 10/500
34/34 [==============================] - 0s 412us/step - loss: 7.8060 - acc

In [15]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 0s 490us/step
[ 0.62460506  2.328727    8.83404    16.118572    2.9156485   5.951101
  4.319941    6.0615544  10.708959   10.262156    1.5895889   1.4489145
  0.01628119  0.02619744]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1634,2,1,13,16,4,3,3,5,8,10,1,1,0,0
1479,2,4,16,15,6,9,3,3,13,19,6,3,0,0
25,2,1,6,23,2,6,4,8,14,11,4,2,0,0
1686,2,1,12,17,4,5,6,8,11,7,3,1,0,0
1454,1,3,9,15,3,7,3,8,8,15,0,2,0,0


### n = 10

In [16]:
n=10
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [17]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(len(X_train.columns), activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
34/34 [==============================] - 0s 509us/step - loss: 27.0220 - accuracy: 0.4113
Epoch 2/500
34/34 [==============================] - 0s 481us/step - loss: 13.4067 - accuracy: 0.4976
Epoch 3/500
34/34 [==============================] - 0s 501us/step - loss: 11.0250 - accuracy: 0.5136
Epoch 4/500
34/34 [==============================] - 0s 443us/step - loss: 9.6123 - accuracy: 0.5378
Epoch 5/500
34/34 [==============================] - 0s 450us/step - loss: 8.8378 - accuracy: 0.5467
Epoch 6/500
34/34 [==============================] - 0s 425us/step - loss: 8.3774 - accuracy: 0.5491
Epoch 7/500
34/34 [==============================] - 0s 443us/step - loss: 8.1178 - accuracy: 0.5538
Epoch 8/500
34/34 [==============================] - 0s 413us/step - loss: 7.9609 - accuracy: 0.5420
Epoch 9/500
34/34 [==============================] - 0s 440us/step - loss: 7.7976 - accuracy: 0.5437
Epoch 10/500
34/34 [==============================] - 0s 443us/step - loss: 7.8257 - acc

In [18]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 0s 622us/step
[ 0.35337526  3.1799393   8.975191   16.397812    2.56419     7.182465
  4.547227    6.0369043   9.223312    8.905641    1.3283552   1.1601696
  0.09176815 -0.0370064 ]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1634,2,1,13,16,4,3,3,5,8,10,1,1,0,0
1479,2,4,16,15,6,9,3,3,13,19,6,3,0,0
25,2,1,6,23,2,6,4,8,14,11,4,2,0,0
1686,2,1,12,17,4,5,6,8,11,7,3,1,0,0
1454,1,3,9,15,3,7,3,8,8,15,0,2,0,0


### n=15

In [19]:
n=15
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [20]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(len(X_train.columns), activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
34/34 [==============================] - 0s 496us/step - loss: 27.1674 - accuracy: 0.4007
Epoch 2/500
34/34 [==============================] - 0s 447us/step - loss: 13.9142 - accuracy: 0.4716
Epoch 3/500
34/34 [==============================] - 0s 505us/step - loss: 11.2993 - accuracy: 0.4775
Epoch 4/500
34/34 [==============================] - 0s 438us/step - loss: 9.9981 - accuracy: 0.4911
Epoch 5/500
34/34 [==============================] - 0s 591us/step - loss: 9.2709 - accuracy: 0.5106
Epoch 6/500
34/34 [==============================] - 0s 517us/step - loss: 8.7824 - accuracy: 0.5313
Epoch 7/500
34/34 [==============================] - 0s 512us/step - loss: 8.4520 - accuracy: 0.5296
Epoch 8/500
34/34 [==============================] - 0s 487us/step - loss: 8.2184 - accuracy: 0.5343
Epoch 9/500
34/34 [==============================] - 0s 477us/step - loss: 8.0595 - accuracy: 0.5443
Epoch 10/500
34/34 [==============================] - 0s 495us/step - loss: 7.9394 - acc

In [21]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 0s 694us/step
[ 0.33033273  3.3477943  11.707733   17.451344    2.9178748   7.6897697
  4.7381835   6.1555448   8.939465    8.124514    1.1674167   1.312059
  0.3542312   0.15888253]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1634,2,1,13,16,4,3,3,5,8,10,1,1,0,0
1479,2,4,16,15,6,9,3,3,13,19,6,3,0,0
25,2,1,6,23,2,6,4,8,14,11,4,2,0,0
1686,2,1,12,17,4,5,6,8,11,7,3,1,0,0
1454,1,3,9,15,3,7,3,8,8,15,0,2,0,0


### n=20

In [22]:
n=20
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [23]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
34/34 [==============================] - 0s 525us/step - loss: 26.2263 - accuracy: 0.4462
Epoch 2/500
34/34 [==============================] - 0s 473us/step - loss: 13.2371 - accuracy: 0.4835
Epoch 3/500
34/34 [==============================] - 0s 2ms/step - loss: 10.7760 - accuracy: 0.4959
Epoch 4/500
34/34 [==============================] - 0s 748us/step - loss: 9.7415 - accuracy: 0.5112
Epoch 5/500
34/34 [==============================] - 0s 889us/step - loss: 9.0670 - accuracy: 0.5236
Epoch 6/500
34/34 [==============================] - 0s 440us/step - loss: 8.6053 - accuracy: 0.5449
Epoch 7/500
34/34 [==============================] - 0s 434us/step - loss: 8.2820 - accuracy: 0.5426
Epoch 8/500
34/34 [==============================] - 0s 437us/step - loss: 8.1104 - accuracy: 0.5437
Epoch 9/500
34/34 [==============================] - 0s 461us/step - loss: 7.9506 - accuracy: 0.5461
Epoch 10/500
34/34 [==============================] - 0s 366us/step - loss: 7.7976 - accur

In [24]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 0s 408us/step
[ 0.13192686  2.9038932   6.558611   19.83437     1.3866603   8.018319
  3.737169    7.422592   13.322785   10.82431     1.7998607   1.0251163
 -0.02559202  0.21237509]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1634,2,1,13,16,4,3,3,5,8,10,1,1,0,0
1479,2,4,16,15,6,9,3,3,13,19,6,3,0,0
25,2,1,6,23,2,6,4,8,14,11,4,2,0,0
1686,2,1,12,17,4,5,6,8,11,7,3,1,0,0
1454,1,3,9,15,3,7,3,8,8,15,0,2,0,0


The above experiments show us the accuracy of each model increases with an increased number of features up to a point, it seem somewhere between 15 and 20 features, where the accuracy on the training data peaks. I will use 15 principal components going forward for the investigations, as this seems to yield the highest accuracy of those tested here

In terms of predicting exact values for the match facts, I am not expecting exact matches between model and the actual results, just hoping for a close approximation, or at the very least, the correct side with the higher value, i.e. home team scores more goals than the away team is correct

## 15 PC model build

#### Splitting data

In [25]:
X = combined[stats_columns]
y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=543)
print(X_train)

         goals   assists  direct_goal_contributions  non_penalty_goals  \
618   0.282813  0.073807                   0.356620           0.269951   
1581  1.264687  0.842364                   2.107051           1.242048   
1600  0.043288 -0.022940                   0.020348           0.069050   
812  -0.163778 -0.007934                  -0.171712          -0.144840   
2113 -0.150000 -0.039141                  -0.189141          -0.137002   
...        ...       ...                        ...                ...   
1542 -0.093617 -0.254741                  -0.348359          -0.078979   
2062 -0.081567  0.127542                   0.045975          -0.044369   
479  -0.660662 -0.268441                  -0.929103          -0.544726   
561  -0.200871  0.054574                  -0.146297          -0.221102   
1785 -0.501214 -0.169302                  -0.670516          -0.484662   

      penalties_scored  penalties_attempted  yellow_cards  red_cards  \
618           0.012862             0.04

#### Scaling X and y

In [26]:
whole_x_scaler = StandardScaler(copy=True).fit(X)
whole_x_train = whole_x_scaler.transform(X)
dump(whole_x_scaler, '../prediction_scaler.bin')


# y_scaler = StandardScaler(copy=True).fit(y_train)

X_scaler = StandardScaler(copy=True).fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)


# y_train = y_scaler.transform(y_train)
# y_test = y_scaler.transform(y_test)

#### Applying PCA

In [27]:
# n=15
# print(X_train)
# pca = PCA(n_components = n, random_state=576)
# pca.fit(X)
# feature_to_pc_map = pd.DataFrame(pca.components_, columns=stats_columns)
# feature_to_pc_map.to_csv("../files/feature_to_15_pcs.csv")

# dump(pca, '../prediction_pca.bin')
# whole_X_pca = pca.transform(whole_x_train)

# pca = PCA(n_components = n, random_state=576)
# pca.fit(X_train)
# X_train = pca.transform(X_train)
# X_test = pca.transform(X_test)


# feature_to_pc_map

The below table shows the breakdown of each principle component in terms of the proportion of the value from each of the table columns used to make it up

In [28]:
df = pd.DataFrame(pca.components_, columns=stats_columns)
df.index.name = "principle_component_number"
df

,goals,assists,direct_goal_contributions,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,non_penalty_expected_goals,...,clean_sheets,penalties_faced,penalties_allowed,penalties_saved,penalties_missed,home_team_at_home_mean_goal_difference,home_team_overall_mean_goal_difference,away_team_at_away_mean_goal_difference,away_team_overall_mean_goal_difference,head_to_head_goal_difference
principle_component_number,,,,,,,,,,,,,,,,,,,,,
0,0.113774,0.125960,0.124219,0.114204,0.067503,0.069344,0.132306,0.067179,0.119271,0.120094,...,0.050092,0.055879,0.053978,0.046351,0.033491,0.006673,0.019182,-0.008737,-0.020654,0.010048
1,-0.108341,-0.087107,-0.105071,-0.107247,-0.073311,-0.076950,0.013237,0.020605,-0.100288,-0.097775,...,0.265032,0.273328,0.261904,0.226938,0.179223,0.011752,0.009510,-0.015483,-0.010714,0.007697
2,0.202537,0.086986,0.165897,0.189020,0.205997,0.208038,-0.093832,-0.062279,0.180007,0.164145,...,0.155262,0.171381,0.165930,0.140269,0.103225,-0.004638,0.008281,-0.005040,-0.006613,-0.009494
3,0.077722,-0.073888,0.020491,0.016399,0.416381,0.403185,0.063660,0.043584,0.074828,0.014778,...,-0.042587,-0.046033,-0.036961,-0.045541,-0.070445,-0.212114,-0.169163,0.162730,0.147332,-0.237649
4,-0.022637,0.050171,0.005562,0.010173,-0.211107,-0.205521,-0.007683,-0.014236,-0.023440,0.008881,...,0.022911,0.025706,0.038543,-0.024770,-0.004912,-0.456012,-0.278414,0.437228,0.264195,-0.510897
5,-0.165465,0.134615,-0.052755,-0.200234,0.107008,0.111622,-0.081601,-0.314774,-0.190525,-0.229676,...,0.033917,0.014824,0.006680,0.047628,0.002970,-0.002653,0.045517,0.071929,0.066960,-0.037169
6,-0.011909,0.024711,0.002249,-0.012000,-0.006790,-0.005914,-0.017838,-0.019332,-0.017668,-0.018552,...,0.000182,-0.005595,-0.011613,0.006353,0.023372,-0.361734,-0.567237,-0.407687,-0.606364,-0.019497
7,0.131587,0.113940,0.130893,0.169485,-0.146680,-0.133791,-0.060934,-0.437499,0.107816,0.141169,...,-0.027321,-0.076557,-0.064223,-0.073002,-0.101401,0.021069,0.192434,-0.016546,-0.121426,-0.083916
8,-0.034019,-0.075770,-0.052478,-0.051972,0.086926,0.085749,0.049404,-0.386531,-0.005432,-0.020436,...,-0.008438,0.024007,0.025153,0.073619,-0.095625,-0.200095,0.460412,0.207967,-0.491725,-0.213783


#### Model Definition

In [29]:
INPUT_SIZE = len(X_train[0])
OUTPUT_SIZE = len(output_columns)

Define

In [30]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(INPUT_SIZE, activation='relu'),
	tf.keras.layers.Dense(12, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE, activation='relu')
])

Compile

In [31]:
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")

Fit

In [32]:
model.fit(X_train, y_train, epochs=500, batch_size=25);

Epoch 1/500


68/68 [==============================] - 0s 544us/step - loss: 25.4400 - accuracy: 0.3274
Epoch 2/500
68/68 [==============================] - 0s 403us/step - loss: 11.8835 - accuracy: 0.4876
Epoch 3/500
68/68 [==============================] - 0s 409us/step - loss: 10.6317 - accuracy: 0.5260
Epoch 4/500
68/68 [==============================] - 0s 367us/step - loss: 10.0395 - accuracy: 0.5426
Epoch 5/500
68/68 [==============================] - 0s 337us/step - loss: 9.9219 - accuracy: 0.5496
Epoch 6/500
68/68 [==============================] - 0s 311us/step - loss: 9.4839 - accuracy: 0.5567
Epoch 7/500
68/68 [==============================] - 0s 333us/step - loss: 9.3479 - accuracy: 0.5579
Epoch 8/500
68/68 [==============================] - 0s 340us/step - loss: 9.2564 - accuracy: 0.5532
Epoch 9/500
68/68 [==============================] - 0s 326us/step - loss: 9.1377 - accuracy: 0.5508
Epoch 10/500
68/68 [==============================] - 0s 396us/step - loss: 7.6420 - accuracy: 0.55

In [33]:
model.evaluate(X_test, y_test)

14/14 [==============================] - 0s 531us/step - loss: 10.8597 - accuracy: 0.4858


[10.85971736907959, 0.4858490526676178]

#### Testing

In [34]:
y_hat = model.predict(X_test)

14/14 [==============================] - 0s 430us/step


In [35]:
total_shots = 0
total_shots_over = 0
total_shots_under = 0

total_shots_on_target = 0
total_shots_on_target_over = 0
total_shots_on_target_under = 0

total_booking_points = 0
total_booking_points_over = 0
total_booking_points_under = 0

correct_score = 0

winner = 0

total_corners = 0
total_corners_over = 0
total_corners_under = 0

total_fouls = 0
total_fouls_over = 0
total_fouls_under = 0

goals_over = 0
goals_under = 0

all_under = 0
all_over = 0

total_tested  = len(y_hat)

In [36]:
for idx, y in enumerate(y_hat):
	home_goals_hat, away_goals_hat, home_shots_hat, away_shots_hat, home_shots_on_target_hat, away_shots_on_target_hat, home_corners_hat, away_corners_hat, home_fouls_hat, away_fouls_hat, home_yellow_cards_hat, away_yellow_cards_hat, home_red_cards_hat, away_red_cards_hat = y
	home_goals, away_goals, home_shots, away_shots, home_shots_on_target, away_shots_on_target, home_corners, away_corners, home_fouls, away_fouls, home_yellow_cards, away_yellow_cards, home_red_cards, away_red_cards = y_test.iloc[idx].tolist()
	
	total_shots += 1 if home_shots_hat+away_shots_hat == home_shots+away_shots else 0
	total_shots_over += 1 if home_shots_hat+away_shots_hat > home_shots+away_shots else 0
	total_shots_under += 1 if home_shots_hat+away_shots_hat < home_shots+away_shots else 0
	
	total_shots_on_target += 1 if home_shots_on_target_hat+away_shots_on_target_hat == home_shots_on_target+away_shots_on_target else 0
	total_shots_on_target_over += 1 if home_shots_on_target_hat+away_shots_on_target_hat > home_shots_on_target+away_shots_on_target else 0
	total_shots_on_target_under += 1 if home_shots_on_target_hat+away_shots_on_target_hat < home_shots_on_target+away_shots_on_target else 0

	total_booking_points += 1 if (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 == (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
	total_booking_points_over += 1 if (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 > (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
	total_booking_points_under += 1 if (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0

	correct_score += 1 if home_goals_hat == home_goals and away_goals_hat == away_goals else 0
	
	winner += 1 if (home_goals_hat > away_goals_hat and home_goals > away_goals) or (home_goals_hat < away_goals_hat and home_goals < away_goals) else 0

	total_fouls += 1 if home_fouls_hat + away_fouls_hat == home_fouls + away_fouls else 0
	total_fouls_over += 1 if home_fouls_hat + away_fouls_hat > home_fouls+ away_fouls else 0
	total_fouls_under += 1 if home_fouls_hat + away_fouls_hat < home_fouls+ away_fouls else 0

	total_corners += 1 if home_corners_hat + away_corners_hat == home_corners + away_corners else 0
	total_corners_over += 1 if home_corners_hat + away_corners_hat > home_corners + away_corners else 0
	total_corners_under += 1 if home_corners_hat + away_corners_hat < home_corners + away_corners else 0

	goals_over += 1 if home_goals_hat + away_goals_hat > home_goals + away_goals else 0
	goals_under += 1 if home_goals_hat + away_goals_hat < home_goals + away_goals else 0

	all_under += 1 if home_shots_hat+away_shots_hat < home_shots+away_shots and home_shots_on_target_hat+away_shots_on_target_hat < home_shots_on_target+away_shots_on_target and (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 and home_corners_hat + away_corners_hat < home_corners + away_corners and home_corners_hat + away_corners_hat < home_corners + away_corners and home_goals_hat + away_goals_hat < home_goals + away_goals else 0
	all_over += 1 if home_shots_hat+away_shots_hat > home_shots+away_shots and home_shots_on_target_hat+away_shots_on_target_hat > home_shots_on_target+away_shots_on_target and (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 > (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 and home_corners_hat + away_corners_hat > home_corners + away_corners and home_goals_hat + away_goals_hat > home_goals + away_goals else 0

In [37]:
print("total_shots: "+str(total_shots/total_tested))
print("total_shots_over: "+str(total_shots_over/total_tested))
print("total_shots_under: "+str(total_shots_under/total_tested))
print("\n")
print("total_shots_on_target: "+str(total_shots_on_target/total_tested))
print("total_shots_on_target_over: "+str(total_shots_on_target_over/total_tested))
print("total_shots_on_target_under: "+str(total_shots_on_target_under/total_tested))
print("\n")
print("total_booking_points: "+str(total_booking_points/total_tested))
print("total_booking_points_over: "+str(total_booking_points_over/total_tested))
print("total_booking_points_under: "+str(total_booking_points_under/total_tested))
print("\n")
print("correct_score: "+str(correct_score/total_tested))
print("\n")
print("winner: "+str(winner/total_tested))
print("\n")
print("total_corners: "+str(total_corners/total_tested))
print("total_corners_over: "+str(total_corners_over/total_tested))
print("total_corners_under: "+str(total_corners_under/total_tested))
print("\n")
print("total_fouls: "+str(total_fouls/total_tested))
print("total_fouls_over: "+str(total_fouls_over/total_tested))
print("total_fouls_under: "+str(total_fouls_under/total_tested))
print("\n")
print("goals_over: "+str(goals_over/total_tested))
print("goals_under: "+str(goals_under/total_tested))
print("\n")
print("all_over: "+str(all_over/total_tested))
print("all_under: "+str(all_under/total_tested))

total_shots: 0.0
total_shots_over: 0.5872641509433962
total_shots_under: 0.41273584905660377


total_shots_on_target: 0.0
total_shots_on_target_over: 0.5849056603773585
total_shots_on_target_under: 0.41509433962264153


total_booking_points: 0.0
total_booking_points_over: 0.5023584905660378
total_booking_points_under: 0.49764150943396224


correct_score: 0.0


winner: 0.5966981132075472


total_corners: 0.0
total_corners_over: 0.535377358490566
total_corners_under: 0.46462264150943394


total_fouls: 0.0
total_fouls_over: 0.5023584905660378
total_fouls_under: 0.49764150943396224


goals_over: 0.5589622641509434
goals_under: 0.4410377358490566


all_over: 0.08490566037735849
all_under: 0.05188679245283019


### Tuning

Create a function to build and compile a model, with the arguments in it being the values to be update during tuning

In [38]:
def get_mlp_model(hidden_layer_one=13, dropout=0.2, learn_rate=0.01, n_h_layers=1):

	model = tf.keras.models.Sequential()

	# input
	model.add(tf.keras.layers.Dense(75, activation="relu", input_dim=75))

	for i in range(n_h_layers):
		model.add(tf.keras.layers.Dense(hidden_layer_one, activation="relu"))

	# dropout layer to remove redundant nodes
	model.add(tf.keras.layers.Dropout(dropout))
	
	# output
	model.add(tf.keras.layers.Dense(14, activation="relu"))

	model.compile(
		optimizer="sgd",
		loss="mse",
		metrics=["accuracy"])
	

	return model

Build a model using the *get_mlp_model* function, letting tensorflow use this function when building the models

In [39]:
model = KerasRegressor(model=get_mlp_model, verbose=0, hidden_layer_one=10, learn_rate=0.01, dropout=0.05, n_h_layers=1);

Define the hyperparameter search space, and then generate the search object to be used later, by RandomizedSearchCV

In [40]:
# define a grid of the hyperparameter search space
hidden_layer_one = [30, 50, 70]
learn_rate = [1e-2, 1e-3, 1e-4]
dropout = [0.1, 0.2, 0.3]
batch_size = [16, 32, 64]
epochs = [100, 200, 500]
n_h_layers = [1, 2, 3, 4]

# create a dictionary from the hyperparameter grid
grid = dict(
	hidden_layer_one=hidden_layer_one,
	learn_rate=learn_rate,
	dropout=dropout,
	batch_size=batch_size,
	epochs=epochs,
	n_h_layers=n_h_layers
)

Build a scoring function for the searcher, average the value of all the *under* stats and return this

In [41]:
def scoring(estimator, test_x: np.ndarray, test_y: pd.DataFrame) -> float:
	test_y = test_y.to_numpy()
	
	# estimator.save("../stats_regression_model.h5")
	y_hat = estimator.predict(test_x)
	
	average_under_rate = 0

	total_shots_under = 0
	total_shots_on_target_under = 0
	total_booking_points_under = 0
	total_corners_under = 0
	total_fouls_under = 0
	goals_under = 0

	total_tested = len(y_hat)
	
	for idx, y in enumerate(y_hat):

		home_goals_hat, away_goals_hat, home_shots_hat, away_shots_hat, home_shots_on_target_hat, away_shots_on_target_hat, home_corners_hat, away_corners_hat, home_fouls_hat, away_fouls_hat, home_yellow_cards_hat, away_yellow_cards_hat, home_red_cards_hat, away_red_cards_hat = y
		home_goals, away_goals, home_shots, away_shots, home_shots_on_target, away_shots_on_target, home_corners, away_corners, home_fouls, away_fouls, home_yellow_cards, away_yellow_cards, home_red_cards, away_red_cards = test_y[idx]

		total_shots_under += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) < home_shots+away_shots else 0
		total_shots_on_target_under += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) < home_shots_on_target+away_shots_on_target else 0
		total_booking_points_under += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
		total_corners_under += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners else 0
		total_fouls_under += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) < home_fouls+ away_fouls else 0
		goals_under += 1 if np.floor(home_goals_hat) + np.floor(away_goals_hat) < home_goals + away_goals else 0

		average_under_rate += ((total_shots_under/total_tested)+(total_shots_on_target_under/total_tested)+(total_booking_points_under/total_tested)+(total_fouls_under/total_tested)+(total_corners_under/total_tested)+(goals_under/total_tested))/6

	average_under_rate = average_under_rate/total_tested
	
	return average_under_rate

Generate the searcher object, used to iterate over the search parameters and determine the best hyper-parameter setup from the space we've created above

In [42]:
searcher = GridSearchCV(estimator=model, n_jobs=-2, param_grid=grid, scoring=scoring, verbose=4, cv=3, refit=True)

searchResults = searcher.fit(X_train, y_train)

bestScore = searchResults.best_score_
bestParams = searchResults.best_params_
print(f"[INFO] best score is {bestScore} using {bestParams}")

Fitting 3 folds for each of 972 candidates, totalling 2916 fits


2024-02-21 01:09:44.325310: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-02-21 01:09:44.746270: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-02-21 01:09:44.781239: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-02-21 01:09:44.844392: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-02-21 01:09:44.865312: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-02-21 01:09:44.867893: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-02-21 01:09:44.869603: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


[CV 1/3] END batch_size=16, dropout=0.1, epochs=100, hidden_layer_one=30, learn_rate=0.01, n_h_layers=1;, score=0.302 total time=  10.7s
[CV 3/3] END batch_size=16, dropout=0.1, epochs=100, hidden_layer_one=30, learn_rate=0.01, n_h_layers=2;, score=0.293 total time=  10.1s
[CV 3/3] END batch_size=16, dropout=0.1, epochs=100, hidden_layer_one=30, learn_rate=0.01, n_h_layers=1;, score=0.275 total time=  10.2s
[CV 2/3] END batch_size=16, dropout=0.1, epochs=100, hidden_layer_one=30, learn_rate=0.01, n_h_layers=1;, score=0.280 total time=  10.3s
[CV 2/3] END batch_size=16, dropout=0.1, epochs=100, hidden_layer_one=30, learn_rate=0.01, n_h_layers=2;, score=0.286 total time=  10.8s
[CV 1/3] END batch_size=16, dropout=0.1, epochs=100, hidden_layer_one=30, learn_rate=0.01, n_h_layers=3;, score=0.320 total time=  10.7s
[CV 1/3] END batch_size=16, dropout=0.1, epochs=100, hidden_layer_one=30, learn_rate=0.01, n_h_layers=2;, score=0.338 total time=  11.1s
[CV 2/3] END batch_size=16, dropout=0.1, 

KeyboardInterrupt: 

In [ ]:
tuned_model = searcher.best_params_
tuned_model

{'batch_size': 64,
 'dropout': 0.1,
 'epochs': 100,
 'hidden_layer_one': 30,
 'learn_rate': 0.01,
 'n_h_layers': 3}

In [ ]:
hidden_layer_one = tuned_model["hidden_layer_one"]
learn_rate = tuned_model["learn_rate"]
dropout = tuned_model["dropout"]
batch_size = tuned_model["batch_size"]
epochs = tuned_model["epochs"]
n_h_layers = tuned_model["n_h_layers"]

model = get_mlp_model(hidden_layer_one=30, learn_rate=0.01, dropout=0.1, n_h_layers=3)
model.fit(X_train, y_train, epochs=500, batch_size=64, )

model.save("../stats_regression_model.h5")
# pd.DataFrame(data=components[:, [p for p in range(15)]], columns=pca.get_feature_names_out(), )

Epoch 1/500
27/27 [==============================] - 0s 939us/step - loss: 33.1961 - accuracy: 0.3020
Epoch 2/500
27/27 [==============================] - 0s 811us/step - loss: 20.4863 - accuracy: 0.4557
Epoch 3/500
27/27 [==============================] - 0s 531us/step - loss: 19.6087 - accuracy: 0.4657
Epoch 4/500
27/27 [==============================] - 0s 607us/step - loss: 19.2338 - accuracy: 0.4622
Epoch 5/500
27/27 [==============================] - 0s 427us/step - loss: 18.8544 - accuracy: 0.4675
Epoch 6/500
27/27 [==============================] - 0s 472us/step - loss: 18.7801 - accuracy: 0.4657
Epoch 7/500
27/27 [==============================] - 0s 551us/step - loss: 18.5783 - accuracy: 0.4687
Epoch 8/500
27/27 [==============================] - 0s 533us/step - loss: 18.3941 - accuracy: 0.4687
Epoch 9/500
27/27 [==============================] - 0s 585us/step - loss: 18.3994 - accuracy: 0.4699
Epoch 10/500
27/27 [==============================] - 0s 580us/step - loss: 18.297

In [ ]:
print(len
	  (X_train))

1692


## Next v  steps

- Tune parameters and layers on the 15 PC model to increase the accuracy
- Create a method to define the accuracy on the test set; strict equality for each of the output values is not necessary as under/over options on the betting platform mean a close approximation of the outcome is all that is required. Success/accuracy measure based on the bet markets available for each game:
	- Total shots
	- Total booking points
	- Correct score
	- Outright
	- Total corners
	- Under/over goals
- Save the model
- Complete the notebook with all annotations and text

RandomSearchCV  
best score is 0.39 using {'n_h_layers': 5, 'learn_rate': 0.0001, 'hidden_layer_one': 15, 'epochs': 100, 'dropout': 0.2, 'batch_size': 64}  
best score is 0.40 using {'n_h_layers': 7, 'learn_rate': 0.0001, 'hidden_layer_one': 15, 'epochs': 1000, 'dropout': 0.75, 'batch_size': 16}  
best score is 0.49 using {'n_h_layers': 3, 'learn_rate': 0.0001, 'hidden_layer_one': 30, 'epochs': 10, 'dropout': 0.3, 'batch_size': 64} 

GridSearchCV  
best score is 0.47 using {'batch_size': 64, 'dropout': 0.4, 'epochs': 100, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 2}  
best score is 0.49100 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 10, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 3}  
best score is 0.49130 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 20, 'hidden_layer_one': 15, 'learn_rate': 1e-05, 'n_h_layers': 4}  
best score is 0.49132 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 20, 'hidden_layer_one': 15, 'learn_rate': 1e-06, 'n_h_layers': 4}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 1e-05, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 1e-07, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 4}  
best score is 0.49138 using {'batch_size': 32, 'dropout': 0.2, 'epochs': 12, 'hidden_layer_one': 10, 'learn_rate': 1e-05, 'n_h_layers': 4}  

- During
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.459 total time=   4.3s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.462 total time=   3.2s
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.430 total time=   3.5s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.458 total time=   5.3s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.412 total time=   3.8s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.451 total time=   3.5s
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.447 total time=   3.8s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.413 total time=   4.8s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.415 total time=   4.9s

## Questions

- Scaling y reduces the accuracy of the model by a factor of 6, why?